- 모델 소스코드 : https://github.com/openvinotoolkit/anomalib/tree/main/src/anomalib/models/fastflow  
- 커스텀 데이터 : https://github.com/openvinotoolkit/anomalib#custom-dataset  
- 노트북 예시 코드 활용 : https://github.com/openvinotoolkit/anomalib/blob/main/notebooks/200_models/201_fastflow.ipynb  

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/2023_이어드림 # working directory
%cd /home/cora3/anomalib/

/home/cora3/anomalib


In [3]:
from pathlib import Path

#dataset_root = Path.cwd() / "/datasets/yeardream/"
dataset_root = '/home/cora3/anomalib/datasets/yeardream'

In [4]:
# !git clone https://github.com/openvinotoolkit/anomalib.git
# %cd anomalib
# %pip install -e .
# %cd ../

In [5]:
# !pip install onnx
# !pip install openvino

In [6]:
from functools import partial, update_wrapper
from types import MethodType
from typing import Any

from matplotlib import pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.optim import Optimizer
from torch.optim.adam import Adam
from torch.utils.data import DataLoader

from anomalib.data import InferenceDataset, TaskType
from anomalib.data.folder import Folder
from anomalib.models.fastflow.lightning_model import Fastflow
from anomalib.models.efficient_ad.lightning_model import EfficientAd
from anomalib.post_processing import (
    NormalizationMethod,
    ThresholdMethod,
    superimpose_anomaly_map,
)
from anomalib.pre_processing.transforms import Denormalize
from anomalib.utils.callbacks import (
    ImageVisualizerCallback,
    MetricsConfigurationCallback,
    MetricVisualizerCallback,
    PostProcessingConfigurationCallback,
)

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.


# Train

In [7]:
task = TaskType.CLASSIFICATION

In [8]:
#dataset_root / 'train'

In [9]:
datamodule = Folder(
    root = dataset_root + '/train',
    normal_dir = 'normal_images/',
    abnormal_dir = 'defect_images_sum/',
    image_size = 256,
    train_batch_size = 16,
    eval_batch_size = 16,
    num_workers = 16,
    task = task,
)

datamodule.setup()
i, data = next(enumerate(datamodule.test_dataloader()))
print(f'Image Shape: {data["image"].shape}')

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/anomalib/data/folder.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'train' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  samples.loc[(samples.label == DirType.NORMAL), "split"] = Split.TRAIN
/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/anomalib/data/folder.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'train' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  samples.loc[(samples.label == DirType.NORMAL), "split"] = Split.TRAIN


Image Shape: torch.Size([16, 3, 256, 256])


In [10]:
model = Fastflow(input_size=(256, 256), backbone="resnet18", flow_steps=8)

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [11]:
model.training = True
train_output = model(data["image"])
hidden_variables, log_jacobian = train_output
print(f"Hidden Variable Shape: {hidden_variables[0].shape}")

Hidden Variable Shape: torch.Size([16, 64, 64, 64])


In [12]:
model.model.training = False
anomaly_map = model(data["image"])
print(f"Anomaly Map Shape: {anomaly_map.shape}")

Anomaly Map Shape: torch.Size([16, 1, 256, 256])


In [13]:
def configure_optimizers(lightning_module: LightningModule, optimizer: Optimizer) -> Any:  # pylint: disable=W0613,W0621
    """Override to customize the LightningModule.configure_optimizers` method."""
    return optimizer


optimizer = Adam(params=model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-5)
fn = partial(configure_optimizers, optimizer=optimizer)
update_wrapper(fn, configure_optimizers)  # necessary for `is_overridden`
model.configure_optimizers = MethodType(fn, model)

In [14]:
callbacks = [
    MetricsConfigurationCallback(
        task=task,
        image_metrics=["AUROC"],
        pixel_metrics=["AUROC"],
    ),
    ModelCheckpoint(
        mode="max",
        monitor="image_AUROC",
    ),
    EarlyStopping(
        monitor="image_AUROC",
        mode="max",
        patience=500,
    ),
    PostProcessingConfigurationCallback(
        normalization_method=NormalizationMethod.MIN_MAX,
        threshold_method=ThresholdMethod.ADAPTIVE,
    ),
    ImageVisualizerCallback(mode="full", task=task, image_save_path="./results/images"),
    MetricVisualizerCallback(mode="full", task=task, image_save_path="./results/images"),
]

In [15]:
trainer = Trainer(
    callbacks=callbacks,
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    max_epochs=500,
    logger=False,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(datamodule=datamodule, model=model)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Cannot perform pixel-level evaluation when task type is classification. Ignoring the following pixel-level metrics: ['AUROC']
/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/cora3/anomalib/checkpoints exists and is not empty.
  rank_zero_warn(f"Che

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


Epoch 0:   1%|          | 2/393 [00:01<03:20,  1.95it/s, loss=2.27e+05, train_loss_step=2.14e+5]

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(


Epoch 3: 100%|██████████| 393/393 [00:21<00:00, 18.48it/s, loss=-1.86e+06, train_loss_step=-1.86e+6, image_AUROC=0.746, train_loss_epoch=-1.63e+6]


In [17]:
trainer.test(datamodule=datamodule, model=model)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Cannot perform pixel-level evaluation when task type is classification. Ignoring the following pixel-level metrics: ['AUROC']
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 53/53 [01:21<00:00,  1.53s/it]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.7300843000411987     │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 0.7300843000411987}]

# Inference

In [18]:
model = Fastflow(input_size=(256, 256), backbone="resnet18", flow_steps=8)
trainer = Trainer(
    # callbacks=callbacks,
    accelerator="auto",  # \<"cpu", "gpu", "tpu", "ipu", "hpu", "auto">,
    devices=1,
    resume_from_checkpoint='/home/cora3/anomalib/checkpoints/epoch=1-step=680.ckpt',
    logger=False,
)

/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:55: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v2.0. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
inference_dataset = InferenceDataset(path=dataset_root + "/test" + "/images/", image_size=(256, 256))
inference_dataloader = DataLoader(dataset=inference_dataset, batch_size=64)

In [20]:
predictions = trainer.predict(model=model, dataloaders=inference_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/cora3/anaconda3/envs/anomalib_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 143/143 [01:23<00:00,  1.72it/s]


In [21]:
predictions[0].keys()

dict_keys(['image', 'image_path', 'anomaly_maps', 'pred_scores', 'pred_labels', 'pred_masks', 'pred_boxes', 'box_scores', 'box_labels'])

In [22]:
predictions[0]['pred_scores']

tensor([-0.3223, -0.0991, -0.1963, -0.2413, -0.2165, -0.1236, -0.1800, -0.1954,
        -0.2357, -0.1366, -0.2286, -0.2103, -0.3041, -0.2299, -0.1262, -0.1384,
        -0.2078, -0.1716, -0.2648, -0.2810, -0.1030, -0.2177, -0.2494, -0.1952,
        -0.2006, -0.2544, -0.1300, -0.2475, -0.2778, -0.1188, -0.2360, -0.2930,
        -0.2609, -0.2646, -0.2204, -0.3289, -0.3127, -0.1278, -0.1965, -0.1970,
        -0.1667, -0.1712, -0.1213, -0.1531, -0.2281, -0.1989, -0.1443, -0.2321,
        -0.2781, -0.1904, -0.2724, -0.2715, -0.1860, -0.1293, -0.2713, -0.3022,
        -0.1914, -0.1503, -0.1925, -0.3094, -0.1921, -0.2252, -0.2638, -0.1549])

In [23]:
len(predictions[0]['pred_labels'])
print([f.split('/')[-1] for f in predictions[0]['image_path']])
print(predictions[0]['pred_labels'])

['test_1548.png', 'test_4921.png', 'test_7079.png', 'test_3752.png', 'test_2252.png', 'test_8900.png', 'test_6996.png', 'test_3813.png', 'test_5443.png', 'test_7921.png', 'test_2840.png', 'test_0116.png', 'test_0128.png', 'test_0092.png', 'test_2942.png', 'test_6853.png', 'test_7424.png', 'test_4936.png', 'test_5428.png', 'test_4134.png', 'test_3386.png', 'test_1427.png', 'test_3462.png', 'test_8721.png', 'test_7167.png', 'test_3116.png', 'test_6079.png', 'test_0438.png', 'test_7014.png', 'test_6052.png', 'test_5640.png', 'test_1542.png', 'test_5990.png', 'test_7350.png', 'test_0595.png', 'test_0113.png', 'test_6743.png', 'test_0722.png', 'test_4581.png', 'test_2902.png', 'test_0138.png', 'test_7399.png', 'test_1331.png', 'test_1323.png', 'test_1070.png', 'test_8916.png', 'test_8017.png', 'test_4445.png', 'test_3751.png', 'test_2443.png', 'test_0966.png', 'test_0425.png', 'test_3670.png', 'test_8689.png', 'test_4743.png', 'test_2420.png', 'test_4234.png', 'test_3682.png', 'test_4415.pn

In [24]:
import pandas as pd

pred = pd.DataFrame(columns = ['ImageId','answer'])
for batch in predictions:
  for i,f in enumerate(batch['image_path']):
    pred.loc[len(pred)] = [f.split('/')[-1], int(batch['pred_labels'][i])]

In [25]:
pred.sort_values(by='ImageId', inplace=True, ignore_index=True)

In [26]:
print(pred['answer'].sum())

0


In [27]:
pred.to_csv('pred_fastflow.csv', index=False)